In [96]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pylab as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.patches as mpatches
from sklearn.model_selection import StratifiedKFold

In [78]:
df = pd.read_csv('data/car_loan.csv')

In [79]:
df['Employment.Type'] = df['Employment.Type'].fillna("Unknown")

In [80]:
a = df['AVERAGE.ACCT.AGE']
a = pd.DataFrame(a)
a = a['AVERAGE.ACCT.AGE'].str.split(' ',expand = True)
a.columns= ['years','months']
a.years = a.years.str[:-3]
a.months = a.months.str[:-3]
a.years = a.years.astype('int')
a.months = a.months.astype('int')
a['AVERAGE.ACCT.AGE'] = a.years*12+a.months
df['AVERAGE.ACCT.AGE']=a['AVERAGE.ACCT.AGE']
#df = pd.concat([df,a.num_mon],axis = 1)
b = df['CREDIT.HISTORY.LENGTH']
b = pd.DataFrame(b)
b = b['CREDIT.HISTORY.LENGTH'].str.split(' ',expand = True)
b.columns= ['years','months']
b.years = b.years.str[:-3]
b.months = b.months.str[:-3]
b.years = b.years.astype('int')
b.months = b.months.astype('int')
b['CREDIT.HISTORY.LENGTH'] = b.years*12+a.months
df['CREDIT.HISTORY.LENGTH']=b['CREDIT.HISTORY.LENGTH']
#df = pd.concat([df,b.Credit_num_mon],axis = 1)

In [81]:
from datetime import date
df['Date.of.Birth'] = pd.to_datetime(df['Date.of.Birth'])
df['DisbursalDate'] = pd.to_datetime(df['DisbursalDate'])
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
df['Date.of.Birth'] = df['Date.of.Birth'].apply(calculate_age)
df['DisbursalDate'] = df['DisbursalDate'].apply(calculate_age)

In [82]:
df['total_overdue_accts'] = df['PRI.OVERDUE.ACCTS']+df['SEC.OVERDUE.ACCTS']
df['total_current_balance'] = df['PRI.CURRENT.BALANCE']+df['SEC.CURRENT.BALANCE']
df['total_sanctioned_amount'] = df['PRI.SANCTIONED.AMOUNT']+df['SEC.SANCTIONED.AMOUNT']
df['total_EMI'] = df['PRIMARY.INSTAL.AMT']+df['SEC.INSTAL.AMT']
df['total_no_of_accts']=df['PRI.NO.OF.ACCTS']+df['SEC.NO.OF.ACCTS']

In [102]:
X_numCol=['disbursed_amount',
'asset_cost',
'ltv','Employee_code_ID',
'PERFORM_CNS.SCORE',
'NEW.ACCTS.IN.LAST.SIX.MONTHS',
'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS','NO.OF_INQUIRIES','Date.of.Birth','DisbursalDate',
'total_overdue_accts','total_current_balance','total_sanctioned_amount',
'total_EMI','total_no_of_accts',
'AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH']
X_catCol=['branch_id',
'supplier_id',
'manufacturer_id',
'Current_pincode_ID','Employment.Type','State_ID','MobileNo_Avl_Flag',
'Aadhar_flag',
'PAN_flag',
'VoterID_flag',
'Driving_flag',
'Passport_flag']
X_age=['Date.of.Birth']
X_order=['PERFORM_CNS.SCORE.DESCRIPTION']

In [84]:
df['ID_provided'] = df.PAN_flag+df.VoterID_flag+df.Driving_flag+df.Passport_flag
df['ID_provided']=df['ID_provided'].apply(lambda x: 1 if x>=1 else 0)
df['ID_provided'].apply(lambda x: 1 if x>=1 else 0).unique()

array([0, 1])

In [100]:
df['PERFORM_CNS.SCORE.DESCRIPTION'].value_counts().index

Index(['No Bureau History Available', 'C-Very Low Risk', 'A-Very Low Risk',
       'D-Very Low Risk', 'B-Very Low Risk', 'M-Very High Risk', 'F-Low Risk',
       'K-High Risk', 'H-Medium Risk', 'E-Low Risk', 'I-Medium Risk',
       'G-Low Risk', 'Not Scored: Sufficient History Not Available',
       'J-High Risk', 'Not Scored: Not Enough Info available on the customer',
       'Not Scored: No Activity seen on the customer (Inactive)',
       'Not Scored: No Updates available in last 36 months',
       'L-Very High Risk', 'Not Scored: Only a Guarantor',
       'Not Scored: More than 50 active Accounts found'],
      dtype='object')

In [85]:
y = df['loan_default']
X = df.loc[:, df.columns != 'loan_default']

In [103]:
# stratified K Fold: variation in balance is very small (4th decimal point)
random_state = 42

# stratified train-test split
X_other, X_test, y_other, y_test = train_test_split(X,y,test_size = 0.1,stratify=y,random_state=random_state)
print('test balance:',y_test.value_counts(normalize=True))
ordinal_cats = [['No Bureau History Available', 'C-Very Low Risk', 'A-Very Low Risk',
       'D-Very Low Risk', 'B-Very Low Risk', 'M-Very High Risk', 'F-Low Risk',
       'K-High Risk', 'H-Medium Risk', 'E-Low Risk', 'I-Medium Risk',
       'G-Low Risk', 'Not Scored: Sufficient History Not Available',
       'J-High Risk', 'Not Scored: Not Enough Info available on the customer',
       'Not Scored: No Activity seen on the customer (Inactive)',
       'Not Scored: No Updates available in last 36 months',
       'L-Very High Risk', 'Not Scored: Only a Guarantor',
       'Not Scored: More than 50 active Accounts found']]
# do StratifiedKFold split on other
kf = StratifiedKFold(n_splits=10,shuffle=True,random_state=random_state)
for train_index, val_index in kf.split(X_other,y_other):
    X_train = X_other.iloc[train_index]
    y_train = y_other.iloc[train_index]
    X_val = X_other.iloc[val_index]
    y_val = y_other.iloc[val_index]
    preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OrdinalEncoder(categories = ordinal_cats), X_order),
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'), X_catCol),
        ('minmax', MinMaxScaler(), X_age),
        ('std', StandardScaler(), X_numCol)])
    clf = Pipeline(steps=[('preprocessor', preprocessor)]) 

    X_train_prep = clf.fit_transform(X_train)
    X_val_prep = clf.transform(X_val)
    X_test_prep = clf.transform(X_test)

test balance: 0    0.782939
1    0.217061
Name: loan_default, dtype: float64


In [104]:
X_train.shape

(188855, 69)

In [45]:
X_test.shape

(23316, 69)

In [46]:
X_val.shape

(20983, 69)